In [1]:
import numpy as np
import scipy.integrate as integrate
import scipy.special as special
from time import perf_counter as clock

from astropy.cosmology import Planck15 as cosmo
from astropy.constants import G, M_sun, c
import astropy.units as u

In [2]:
#Lens parameters

z_lens = 0.035
z_source = 2.1

D_l = cosmo.angular_diameter_distance(z_lens)
D_s = cosmo.angular_diameter_distance(z_source)
D_ls = cosmo.angular_diameter_distance_z1z2(z_lens, z_source)

In [3]:
#Useful constants
metre2Mpc = (1*u.m).to(u.Mpc)/u.m           #Constant factor to convert metre to Mpc.
kg2Msun = (1*u.kg/M_sun)*u.solMass/u.kg     #Constant factor to convert kg to Msun

G_Mpc = G*(metre2Mpc)**3/kg2Msun            #Gravitational constant in Mpc³/(Msun s²)
c_Mpc = c*metre2Mpc                         #Speed of light in Mpc/s

In [4]:
#Define as funções que irão calcular os angulos de deflexão
#Atenção na sum, ela deve incluir a exponencial!!!!!!!!!!!!!!!!!!!!!!!!!!!

def alpha_x(tau, y, x, M, sigma,q):
    
    x_til = x/sigma
    y_til = y/sigma
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(x_til/(np.sqrt(1 - eta_sq*(tau)**2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    
    '''
    sum = np.sum(aux)
    print(sum)
    print('Sem unidade')
    sum = np.sum(aux)/(aux.unit)
    print(sum)
    print('################################')
    '''
    sum = np.sum(aux*exp)/(aux.unit)
    function = tau*sum
    
    return function

def alpha_y(tau, y, x, M, sigma,q):
    
    x_til = x/sigma
    y_til = y/sigma
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(y_til/(np.power(1 - eta_sq*(tau)**2, 3/2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    
    sum = np.sum(aux*exp)/(aux.unit)
    function = tau*sum
    
    return function

In [5]:
#MGE parameters


#Reading data
Surf_mge, sigma, qObs = np.loadtxt('/home/carlos/autolens_workspace/howtolens/MGE/SIS MGE.txt', unpack=True)

#Converting and computing some quantities
Surf_mge = Surf_mge*(u.solLum/u.pc**2)                         #Surface Luminosity in L_sun/pc²
sigma = sigma*u.arcsec                                         #Sigma in arcsec
sigma_pc = (sigma*D_l).to(u.pc, u.dimensionless_angles())      #Convert sigma in arcsec to sigma in pc
Lum_mge = 2*np.pi*Surf_mge*(sigma_pc**2)*qObs                  #Total Luminosity per gaussian component




#Defining inputs for the integral bellow
ML = 2.75*(u.solMass/u.solLum)                                 #Mass-to-light ratio in kg/L_sun
i = np.deg2rad(80)*u.rad                                       #Inclination angle in rad
M0 = ML*Lum_mge                                                #Mass per gaussian component in kg
q0 = np.sqrt(qObs**2 - np.cos(i)**2)/np.sin(i)                 #Deprojected axial ratio
sigma0 = (sigma).to(u.rad)                                     #Sigma per gaussian in rad



In [6]:
#Lens Constants for integral

critical_density = (c_Mpc**2/(4*np.pi*G_Mpc))*(D_s/(D_ls*D_l))
const_factor = 1/(np.pi*critical_density*D_l**2)
const_factor.value

1.253233523498839e-21

In [7]:
time = clock()
x = -0.45*u.arcsec
y = 0.45*u.arcsec

xrad = x.to(u.rad)
yrad = y.to(u.rad)

xbin = const_factor*(integrate.quad(alpha_x, 0, 1, args=(yrad, xrad, M0, sigma0, q0))[0])*(u.solMass*u.rad)
ybin = const_factor*(integrate.quad(alpha_y, 0, 1, args=(yrad, xrad, M0, sigma0, q0))[0])*(u.solMass*u.rad)

print(xbin.to(u.arcsec), ybin.to(u.arcsec))

print(clock() - time)

-580273.0633568922 arcsec 580273.0633568922 arcsec
0.5089693110021472


In [8]:
print(xrad, yrad)

-2.181661564992912e-06 rad 2.181661564992912e-06 rad


In [9]:
import autolens as al
import astropy.units as u
import numpy as np

In [12]:
mge = al.mp.MGE(centre=(0,0))
mge.MGE_comps(M=M0.value, sigma=sigma0.value, q=q0.value, z_l=z_lens, z_s=z_source)

grid = al.Grid.uniform(shape_2d=(10, 10), pixel_scales=0.1, sub_size=1)
grid[0]

Grid([ 0.45, -0.45])

In [13]:
mge.deflections_from_grid(grid)[0]

Grid([ 580273.06335689, -580273.06335689])